# ViT 2: Electric Transformaroo
We learned a few lessons from the previous experiment. First, and primarily, a larger model doesn't always mean better. We used the large imageNet transformer model, but that didn't actually lead to good predictions. Also, I think I screwed up a bunch of stuff, and the large model is taking way too long to train for me to iterate at all. We are going to switch to a smaller model. We are going to follow the ViTMAE method, use masked auto encodings for pretraining in a self-supervised manner on the entire training set, then fine tune on the prediction task. Guide found here: https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-pretraining/run_mae.py 

This notebook has the fine-tuning process.

In [2]:
# imports
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTMAEForPreTraining, ViTMAEConfig
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

True


In [3]:
from dataloader import *
from utils import *
from trainer import *

In [9]:
# CONSTANTS
FEATURE_EXTRACTOR_NAME = 'facebook/vit-mae-base'
VIT_MODEL_NAME = 'vit-mae-chexpert/'
TRAIN_SPLIT = 0.8
BATCH_SIZE = 48
LEARNING_RATE = 1.5e-3
LR_SCHEDULER_TYPE = "cosine"
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.05
LOGGING_STRATEGY = "steps"
LOGGING_STEPS = 10
FP16 = True
EPOCHS = 10
EVALUATION_STRATEGY = "steps"
EVAL_STEPS = 200
OUTPUT_DIR = './vit-mae-chexpert-fine-tuned-3'
REMOVE_UNUSED_COLUMNS = False
GRAD_ACCUM_STEPS = 5
DATALOADER_NUM_WORKERS = 4

In [10]:
feature_extractor = ViTFeatureExtractor.from_pretrained(FEATURE_EXTRACTOR_NAME, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

In [11]:
# set up our transforms
transforms = [
    RandomResizedCrop(feature_extractor.size, scale=(0.2, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
]

In [17]:
np.random.seed(42)
train_df = pd.read_csv("ChexPert/train.csv")
train_df, eval_df = train_test_split(train_df, train_size=TRAIN_SPLIT)

train_dataset = ChexpertViTDataset("ChexPert/data", train_df, feature_extractor, transforms=transforms, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)
eval_dataset = ChexpertViTDataset("ChexPert/data", eval_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85)

In [7]:
train_dataset[0]

{'pixel_values': tensor([[[-0.8335, -0.8678, -0.8678,  ..., -0.2856,  0.0569,  0.1768],
         [-0.7137, -0.8335, -0.8849,  ..., -0.6794, -0.6109, -0.4054],
         [-0.7993, -0.8164, -0.6965,  ..., -0.6452, -0.5082, -0.6281],
         ...,
         [ 0.8276,  0.8618,  0.8447,  ...,  1.6667,  1.7009,  1.6838],
         [ 0.7248,  0.8276,  0.8789,  ...,  1.7352,  1.6153,  1.7352],
         [ 0.7419,  0.7933,  0.8104,  ...,  1.8208,  1.7009,  1.7352]],

        [[-0.7227, -0.7577, -0.7577,  ..., -0.1625,  0.1877,  0.3102],
         [-0.6001, -0.7227, -0.7752,  ..., -0.5651, -0.4951, -0.2850],
         [-0.6877, -0.7052, -0.5826,  ..., -0.5301, -0.3901, -0.5126],
         ...,
         [ 0.9755,  1.0105,  0.9930,  ...,  1.8333,  1.8683,  1.8508],
         [ 0.8704,  0.9755,  1.0280,  ...,  1.9034,  1.7808,  1.9034],
         [ 0.8880,  0.9405,  0.9580,  ...,  1.9909,  1.8683,  1.9034]],

        [[-0.4973, -0.5321, -0.5321,  ...,  0.0605,  0.4091,  0.5311],
         [-0.3753, -0.4973, 

In [ ]:
model = ViTForImageClassification.from_pretrained(
    VIT_MODEL_NAME,
    num_labels=len(train_dataset.labels)
).to("cuda")

In [18]:
# set up training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_steps = EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    report_to="tensorboard",
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_strategy=LOGGING_STRATEGY,
    logging_steps=LOGGING_STEPS,
    dataloader_num_workers=DATALOADER_NUM_WORKERS
)

PyTorch: setting up devices


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using amp half precision backend


In [20]:
trainer.train()

***** Running training *****
  Num examples = 152878
  Num Epochs = 10
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 240
  Gradient Accumulation steps = 5
  Total optimization steps = 6370


Step,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion
10,0.552000,0.534397,0.236179,0.000697,0.563012,0.589833,0.583948,0.715736,0.669734


***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48


KeyboardInterrupt: 

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 38149
  Batch size = 48


{'eval_loss': 0.40164998173713684,
 'eval_accuracy': 0.3934310204723584,
 'eval_runtime': 196.3203,
 'eval_samples_per_second': 194.32,
 'eval_steps_per_second': 4.05,
 'epoch': 3.0}

In [18]:
trainer.save_state()

In [19]:
trainer.save_model()

Saving model checkpoint to ./vit-mae-chexpert-fine-tuned
Configuration saved in ./vit-mae-chexpert-fine-tuned/config.json
Model weights saved in ./vit-mae-chexpert-fine-tuned/pytorch_model.bin


TypeError: save_metrics() missing 2 required positional arguments: 'split' and 'metrics'